In [1]:
import glob
import json
from datetime import datetime
import pandas as pd
import numpy as np

In [2]:
# Path pattern to locate all JSON files across snapshots
file_pattern = "C:/Users/Dell/Desktop/AI for Software Engineering/Project/Dataset/snapshot_*/*_pr_sharings.json"

# Initialize an empty list to store all JSON data
all_data = []

# Loop through all matching file paths and load each file
for path in glob.glob(file_pattern):
    print(path)
    with open(path, "r") as file:
        data = json.load(file)
        all_data.append(data)

# all_data now contains data from all JSON files
print(f"Total files read: {len(all_data)}")

C:/Users/Dell/Desktop/AI for Software Engineering/Project/Dataset\snapshot_20230727\20230727_195927_pr_sharings.json
C:/Users/Dell/Desktop/AI for Software Engineering/Project/Dataset\snapshot_20230803\20230803_093947_pr_sharings.json
C:/Users/Dell/Desktop/AI for Software Engineering/Project/Dataset\snapshot_20230810\20230810_123110_pr_sharings.json
C:/Users/Dell/Desktop/AI for Software Engineering/Project/Dataset\snapshot_20230817\20230817_125147_pr_sharings.json
C:/Users/Dell/Desktop/AI for Software Engineering/Project/Dataset\snapshot_20230824\20230824_100450_pr_sharings.json
C:/Users/Dell/Desktop/AI for Software Engineering/Project/Dataset\snapshot_20230831\20230831_060603_pr_sharings.json
C:/Users/Dell/Desktop/AI for Software Engineering/Project/Dataset\snapshot_20230907\20230907_091631_pr_sharings.json
C:/Users/Dell/Desktop/AI for Software Engineering/Project/Dataset\snapshot_20230914\20230914_074826_pr_sharings.json
C:/Users/Dell/Desktop/AI for Software Engineering/Project/Datase

In [3]:
# Initialize a list for rows that will form the DataFrame
rows = []

# Define the pattern analysis function
def analyze_prompt_structure(data):
    detected_patterns = []
    
    persona_keywords = ["you are", "act as", "pretend to be", "pretend you are"]
    recipe_keywords = ["step-by-step", "recipe", "guide"]
    template_keywords = ["template", "formatting"]
    automator_keywords = ["script", "code", "executable"]
    simple_instruction_keywords = ["explain", "describe", "list", "tell me", "give me"]
    context_instruction_keywords = ["based on", "with this information"]
    question_keywords = ["what", "where", "when", "who", "why"]

    def contains_keywords(text, keywords):
        return any(keyword in text.lower() for keyword in keywords)

    for source in data.get("Sources", []):
        body = source.get("Body", "")
        created_at = source.get("CreatedAt")
        closed_at = source.get("ClosedAt")
        state = "Closed" if closed_at is not None else "Open"

        time_lapsed = None
        if created_at and closed_at:
            created_at_dt = datetime.strptime(created_at, "%Y-%m-%dT%H:%M:%SZ")
            closed_at_dt = datetime.strptime(closed_at, "%Y-%m-%dT%H:%M:%SZ")
            time_lapsed = closed_at_dt - created_at_dt

        body_patterns = []

        # Check for patterns in the body text
        if contains_keywords(body, persona_keywords):
            body_patterns.append("Persona Pattern")
        if contains_keywords(body, recipe_keywords):
            body_patterns.append("Recipe Pattern")
        if contains_keywords(body, template_keywords):
            body_patterns.append("Template Pattern")
        if contains_keywords(body, automator_keywords):
            body_patterns.append("Output Automator Pattern")
        if contains_keywords(body, simple_instruction_keywords):
            body_patterns.append("Simple Instruction Pattern")
        if contains_keywords(body, context_instruction_keywords):
            body_patterns.append("Context and Instruction Pattern")
        if contains_keywords(body, question_keywords):
            body_patterns.append("Question Pattern")

        conversations = source.get("ChatgptSharing", [])
        for item in conversations:
            number_of_prompts = item.get("NumberOfPrompts", "N/A")
            conversation_url = item.get("URL", "N/A")

            for pattern in body_patterns:
                detected_patterns.append((
                    source['Number'],
                    pattern,
                    state,
                    time_lapsed,
                    number_of_prompts,
                    conversation_url
                ))

    return detected_patterns

In [14]:
# Loop through each dictionary in all_data
for data_dict in all_data:
    # Check if 'Sources' key exists in each dictionary
    if 'Sources' in data_dict:
        for item in data_dict['Sources']:
            if 'ChatgptSharing' in item:
                for sharing in item['ChatgptSharing']:
                    if 'Conversations' in sharing:
                        for conversation in sharing['Conversations']:
                            # Adding the fields to row
                            row = {
                                "Type": item.get('Type'),
                                "URL": item.get('URL'),
                                "Author": item.get('Author'),
                                "RepoName": item.get('RepoName'),
                                "RepoLanguage": item.get('RepoLanguage'),
                                "Number": item.get('Number'),
                                "Title": item.get('Title'),
                                "Body": item.get('Body'),
                                "MergedAt": item.get('MergedAt'),
                                "UpdatedAt": item.get('UpdatedAt'),
                                "State": item.get('State'),
                                "Additions": item.get('Additions'),
                                "Deletions": item.get('Deletions'),
                                "ChangedFiles": item.get('ChangedFiles'),
                                "CommitsTotalCount": item.get('CommitsTotalCount'),
                                "CSharing_URL": sharing.get('URL'),
                                "CSharing_Status": sharing.get('Status'),
                                "CSharing_DateOfConversation": sharing.get('DateOfConversation'),
                                "CSharing_Title": sharing.get('Title'),
                                "CSharing_NumberOfPrompts": sharing.get('NumberOfPrompts'),
                                "CSharing_TokensOfPrompts": sharing.get('TokensOfPrompts'),
                                "CSharing_TokensOfAnswers": sharing.get('TokensOfAnswers'),
                                "Conversation_Prompt": conversation.get('Prompt'),
                                "Conversation_Answer": conversation.get('Answer')
                            }
                            rows.append(row)

# Create DataFrame from rows
df = pd.DataFrame(rows)

In [15]:
df['State'].unique()

array(['OPEN', 'MERGED', 'CLOSED'], dtype=object)

In [5]:
# Detect patterns for each source and add to the DataFrame
detected_patterns = analyze_prompt_structure(data_dict)  # Use the last data_dict for patterns

In [6]:
# Prepare a dictionary to store detected patterns for easy lookup
pattern_dict = {}
for number, pattern, state, time_lapsed, number_of_prompts, conversation_url in detected_patterns:
    if number not in pattern_dict:
        pattern_dict[number] = []
    pattern_dict[number].append(pattern)

# Add a new column for detected patterns
df['Detected Patterns'] = df['Number'].map(lambda x: ', '.join(pattern_dict.get(x, [])))

In [7]:
df.head()

,Type,URL,Author,RepoName,RepoLanguage,Number,Title,Body,MergedAt,UpdatedAt,...,CSharing_URL,CSharing_Status,CSharing_DateOfConversation,CSharing_Title,CSharing_NumberOfPrompts,CSharing_TokensOfPrompts,CSharing_TokensOfAnswers,Conversation_Prompt,Conversation_Answer,Detected Patterns
0,pull request,https://github.com/labdao/plex/pull/469,AdamGoyer,labdao/plex,Go,469,add readme for openbabel to PLEX,The Chatgpt Thread used to create this pull re...,None,2023-07-05T03:30:59Z,...,https://chat.openai.com/share/8bd33825-e8c6-44...,200,"July 5, 2023",Open Babel on PLEX,6,2895,2311,"Good evening Chatgpt,\nI'd like your help to w...",Thanks for sharing the README file for Open Ba...,
1,pull request,https://github.com/labdao/plex/pull/469,AdamGoyer,labdao/plex,Go,469,add readme for openbabel to PLEX,The Chatgpt Thread used to create this pull re...,None,2023-07-05T03:30:59Z,...,https://chat.openai.com/share/8bd33825-e8c6-44...,200,"July 5, 2023",Open Babel on PLEX,6,2895,2311,"Here is the PLEX readme, I think this will ans...",Thanks for sharing the README file for the PLE...,
2,pull request,https://github.com/labdao/plex/pull/469,AdamGoyer,labdao/plex,Go,469,add readme for openbabel to PLEX,The Chatgpt Thread used to create this pull re...,None,2023-07-05T03:30:59Z,...,https://chat.openai.com/share/8bd33825-e8c6-44...,200,"July 5, 2023",Open Babel on PLEX,6,2895,2311,Essencially PLEX loads an openbabel docker con...,"Based on the given information, it looks like ...",
3,pull request,https://github.com/labdao/plex/pull/469,AdamGoyer,labdao/plex,Go,469,add readme for openbabel to PLEX,The Chatgpt Thread used to create this pull re...,None,2023-07-05T03:30:59Z,...,https://chat.openai.com/share/8bd33825-e8c6-44...,200,"July 5, 2023",Open Babel on PLEX,6,2895,2311,"Excellent work chat, I think we should also ex...",These JSON files are indeed configuration file...,
4,pull request,https://github.com/labdao/plex/pull/469,AdamGoyer,labdao/plex,Go,469,add readme for openbabel to PLEX,The Chatgpt Thread used to create this pull re...,None,2023-07-05T03:30:59Z,...,https://chat.openai.com/share/8bd33825-e8c6-44...,200,"July 5, 2023",Open Babel on PLEX,6,2895,2311,"Noted,\nTake a look at this docker file, and t...",This Dockerfile is used to create a Docker ima...,


In [8]:
print((df['Detected Patterns'] == '').sum())
print(df['Detected Patterns'].isna().sum())
df.shape

2938
0


(7976, 25)

In [9]:
df_to_save = df[(df['Detected Patterns'] != '')]
df_to_save.shape

(5038, 25)

In [10]:
# Save the DataFrame to a CSV file
df_to_save.to_csv('pr_sharing_visualize_output_with_patterns.csv', index=False)

In [11]:
df_to_save.head()

,Type,URL,Author,RepoName,RepoLanguage,Number,Title,Body,MergedAt,UpdatedAt,...,CSharing_URL,CSharing_Status,CSharing_DateOfConversation,CSharing_Title,CSharing_NumberOfPrompts,CSharing_TokensOfPrompts,CSharing_TokensOfAnswers,Conversation_Prompt,Conversation_Answer,Detected Patterns
149,pull request,https://github.com/IararIV/karaok.ai/pull/1,IararIV,IararIV/karaok.ai,None,1,"initial interface, incorrect credentials if we...",already gets top k matches for tracks\r\nto ad...,2023-07-16T12:34:50Z,2023-07-16T12:34:51Z,...,https://chat.openai.com/share/d0ce9b4e-2063-43...,200,"July 16, 2023",Spotify Auth Flows,2,38,683,What is the difference between SpotifyClientCr...,"In the context of the Spotify API, SpotifyClie...","Output Automator Pattern, Output Automator Pat..."
150,pull request,https://github.com/IararIV/karaok.ai/pull/1,IararIV,IararIV/karaok.ai,None,1,"initial interface, incorrect credentials if we...",already gets top k matches for tracks\r\nto ad...,2023-07-16T12:34:50Z,2023-07-16T12:34:51Z,...,https://chat.openai.com/share/d0ce9b4e-2063-43...,200,"July 16, 2023",Spotify Auth Flows,2,38,683,If I want to add a track from the id to my use...,If you want to add a track from its ID to your...,"Output Automator Pattern, Output Automator Pat..."
199,pull request,https://github.com/PYT-Acadamy/PYT-Acadamy.git...,david-star-git,PYT-Acadamy/PYT-Acadamy.github.io,HTML,20,git und github tutorial,ich habe 'https://youtubetranscript.com/?v=mJ-...,2023-07-22T14:08:36Z,2023-07-22T14:08:36Z,...,https://chat.openai.com/share/6a1f6db3-3ea1-44...,200,"July 21, 2023",Simple Git and GitHub.,3,3647,2192,every resource about git or github\nif you loo...,Title: A Simple Guide to Git and GitHub for Be...,Output Automator Pattern
200,pull request,https://github.com/PYT-Acadamy/PYT-Acadamy.git...,david-star-git,PYT-Acadamy/PYT-Acadamy.github.io,HTML,20,git und github tutorial,ich habe 'https://youtubetranscript.com/?v=mJ-...,2023-07-22T14:08:36Z,2023-07-22T14:08:36Z,...,https://chat.openai.com/share/6a1f6db3-3ea1-44...,200,"July 21, 2023",Simple Git and GitHub.,3,3647,2192,use the words he used to describe things,"Title: A Simple Guide to Git and GitHub, Expla...",Output Automator Pattern
201,pull request,https://github.com/PYT-Acadamy/PYT-Acadamy.git...,david-star-git,PYT-Acadamy/PYT-Acadamy.github.io,HTML,20,git und github tutorial,ich habe 'https://youtubetranscript.com/?v=mJ-...,2023-07-22T14:08:36Z,2023-07-22T14:08:36Z,...,https://chat.openai.com/share/6a1f6db3-3ea1-44...,200,"July 21, 2023",Simple Git and GitHub.,3,3647,2192,übersetze das auf deutsch,Titel: Ein einfacher Leitfaden zu Git und GitH...,Output Automator Pattern


In [12]:
df_to_save['State'].unique()

array(['MERGED', 'OPEN', 'CLOSED'], dtype=object)